In [38]:
import numpy as np
import warnings
from utils import *
from calibrators import * 
import pickle
warnings.filterwarnings("ignore")
import torch
import sys
sys.path.append('../')
import time
import copy
from SLURM.sklearn_config import *
from SLURM.pytorch_config import *



from SLURM.pytorchShuffleFashion import *

In [39]:
%run ./SLURM/pytorch_config.py

In [40]:


def preprosses_Fashion(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    train_mean = X_train.mean() / 255.
    train_std = X_train.std() / 255.

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])
    # Also use X_train in normalize since train/val sets should have same distribution
    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    trainset = Fashion_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = Fashion_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = Fashion_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data

In [41]:

def main(dataset_name,shuffle_num):
    train_loader, val_loader, test_loader, data = preprosses_Fashion(dataset_name, shuffle_num)
    net = Fashion_net()
    num_epochs = 30
    learning_rate = 0.001
    # CrossEntropyLoss - Loss function
    criterion = nn.CrossEntropyLoss()  # loss function
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    if torch.cuda.is_available():
        net = net.cuda()
    print(net)
    print('=================================================================')

    PATH_model = f'./{dataset_name}/{shuffle_num}/pytorch/'
    net.load_state_dict(torch.load(PATH_model + 'model.pth'))

    # calc predictions
    Y_pred_test = model_testing(net, criterion, test_loader, 'test')
    Y_pred_val = model_testing(net, criterion, val_loader, 'val')
    Y_pred_train = model_testing(net, criterion, train_loader, 'train')
    np.save(PATH_model +"y_pred_test.npy",Y_pred_test)
    np.save(PATH_model +"y_pred_val.npy",Y_pred_val)
    np.save(PATH_model +"y_pred_train.npy",Y_pred_train)
    
    # calc logits
    logits_val = compute_logits(net, val_loader)
    logits_test = compute_logits(net, test_loader)
    logits_train = compute_logits(net, train_loader)
    # predict proba (after softmax)
    val_proba = softmax(logits_val, axis=1)
    test_proba = softmax(logits_test, axis=1)
    train_proba = softmax(logits_train, axis=1)
    np.save(PATH_model +"all_predictions_val.npy",val_proba)
    np.save(PATH_model +"all_predictions_test.npy",test_proba)
    np.save(PATH_model +"all_predictions_train.npy",train_proba)

In [42]:
for i in range(10):
    main('Fashion',i)

Fashion_net(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)
   testing progress: 98.17%  ......  loss: 0.7473 , acc: 0.8955

 test of the network on the 14000 test images: 89.53571428571429%
   testing progress: 98.17%  ......  loss: 0.7356 , acc: 0

In [58]:
from torchvision.models import resnet50


def preprosses_GTSRB(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 30, 30, 3))])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 30, 30, 3))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_test.reshape(-1, 30, 30, 3))
    ])

    trainset = GTSRB_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = GTSRB_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = GTSRB_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data


def preprosses_GTSRB(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 30, 30, 3))])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 30, 30, 3))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_test.reshape(-1, 30, 30, 3))
    ])

    trainset = GTSRB_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = GTSRB_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = GTSRB_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data



In [59]:

def main(dataset_name, shuffle_num):
    train_loader, val_loader, test_loader, data = preprosses_GTSRB(dataset_name, shuffle_num)
    resn50 = resnet50(pretrained=True, progress=True)
    net = GTSRB_net(resn50)

    num_epochs = 30
    learning_rate = 0.001

    if torch.cuda.is_available():
        net = net.cuda()
    print(net)
    print('=================================================================')

    # CrossEntropyLoss - Loss function
    criterion = nn.CrossEntropyLoss()  # loss function
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    PATH_model = f'./{dataset_name}/{shuffle_num}/pytorch/'
    # training and validating
    
    print('preload state_dict')
    net.load_state_dict(torch.load(PATH_model + 'model.pth'))

    
        # calc predictions
    Y_pred_test = model_testing(net, criterion, test_loader, 'test')
    Y_pred_val = model_testing(net, criterion, val_loader, 'val')
    Y_pred_train = model_testing(net, criterion, train_loader, 'train')
    np.save(PATH_model +"y_pred_test.npy",Y_pred_test)
    np.save(PATH_model +"y_pred_val.npy",Y_pred_val)
    np.save(PATH_model +"y_pred_train.npy",Y_pred_train)
        # calc logits
    logits_val = compute_logits(net, val_loader)
    logits_test = compute_logits(net, test_loader)
    logits_train = compute_logits(net, train_loader)
    # predict proba (after softmax)
    val_proba = softmax(logits_val, axis=1)
    test_proba = softmax(logits_test, axis=1)
    train_proba = softmax(logits_train, axis=1)
    np.save(PATH_model +"all_predictions_val.npy",val_proba)
    np.save(PATH_model +"all_predictions_test.npy",test_proba)
    np.save(PATH_model +"all_predictions_train.npy",train_proba)
   



In [60]:
for i in range(10):
    main('GTSRB_RGB',i)

GTSRB_net(
  (rn50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [49]:
from SLURM.pytorchShuffleCIFAR10 import *
from SLURM.pytorch_config import *


def preprosses_CIFAR(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 32, 32, 3))])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_train.reshape(-1, 32, 32, 3))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_dataset(X_test.reshape(-1, 32, 32, 3))
    ])

    trainset = CIFAR_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = CIFAR_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = CIFAR_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data

In [50]:

def main(dataset_name, shuffle_num):
    train_loader, val_loader, test_loader, data = preprosses_CIFAR(dataset_name, shuffle_num)
    net = CIFAR_net()
    num_epochs = 30
    learning_rate = 0.001

    if torch.cuda.is_available():
        net = net.cuda()
    
    # CrossEntropyLoss - Loss function
    criterion = nn.CrossEntropyLoss()  # loss function
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    PATH_model = f'./{dataset_name}/{shuffle_num}/pytorch/'
    # training and validating
    
    print('preload state_dict')
    net.load_state_dict(torch.load(PATH_model + 'model.pth'))

    
        # calc predictions
    Y_pred_test = model_testing(net, criterion, test_loader, 'test')
    Y_pred_val = model_testing(net, criterion, val_loader, 'val')
    Y_pred_train = model_testing(net, criterion, train_loader, 'train')
    np.save(PATH_model +"y_pred_test.npy",Y_pred_test)
    np.save(PATH_model +"y_pred_val.npy",Y_pred_val)
    np.save(PATH_model +"y_pred_train.npy",Y_pred_train)   
    # calc logits
    logits_val = compute_logits(net, val_loader)
    logits_test = compute_logits(net, test_loader)
    logits_train = compute_logits(net, train_loader)
    # predict proba (after softmax)
    val_proba = softmax(logits_val, axis=1)
    test_proba = softmax(logits_test, axis=1)
    train_proba = softmax(logits_train, axis=1)
    np.save(PATH_model +"all_predictions_val.npy",val_proba)
    np.save(PATH_model +"all_predictions_test.npy",test_proba)
    np.save(PATH_model +"all_predictions_train.npy",train_proba)


In [51]:
for i in range(10):
    main('CIFAR_RGB',i)

preload state_dict
   testing progress: 98.40%  ......  loss: 2.1313 , acc: 0.6716

 test of the network on the 12000 test images: 67.14166666666667%
   testing progress: 98.40%  ......  loss: 2.1684 , acc: 0.6719

 val of the network on the 12000 test images: 67.10833333333333%
   testing progress: 99.82%  ......  loss: 0.1217 , acc: 0.9631

 train of the network on the 36000 test images: 96.31388888888888%
preload state_dict
   testing progress: 98.40%  ......  loss: 2.1930 , acc: 0.6736

 test of the network on the 12000 test images: 67.39166666666667%
   testing progress: 98.40%  ......  loss: 2.1855 , acc: 0.6751

 val of the network on the 12000 test images: 67.55833333333334%
   testing progress: 99.82%  ......  loss: 0.1221 , acc: 0.9631

 train of the network on the 36000 test images: 96.31111111111112%
preload state_dict
   testing progress: 98.40%  ......  loss: 2.1647 , acc: 0.6606

 test of the network on the 12000 test images: 66.04166666666667%
   testing progress: 98.40

In [43]:

def preprosses_SignLanguage(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    train_mean = X_train.mean() / 255.
    train_std = X_train.std() / 255.

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])
    # Also use X_train in normalize since train/val sets should have same distribution
    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    trainset = SignLanguage_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = SignLanguage_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = SignLanguage_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data




In [44]:

def main(dataset_name, shuffle_num):
    train_loader, val_loader, test_loader, data = preprosses_SignLanguage(dataset_name, shuffle_num)
    net = SignLanguage_net()

    num_epochs = 30
    learning_rate = 0.001

    if torch.cuda.is_available():
        net = net.cuda()
    print(net)
    print('=================================================================')

    # CrossEntropyLoss - Loss function
    criterion = nn.CrossEntropyLoss()  # loss function
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    PATH_model = f'./{dataset_name}/{shuffle_num}/pytorch/'


    print('preload state_dict')
    net.load_state_dict(torch.load(PATH_model + 'model.pth'))

    # calc predictions
    Y_pred_test = model_testing(net, criterion, test_loader, 'test')
    Y_pred_val = model_testing(net, criterion, val_loader, 'val')
    Y_pred_train = model_testing(net, criterion, train_loader, 'train')
    np.save(PATH_model +"y_pred_test.npy",Y_pred_test)
    np.save(PATH_model +"y_pred_val.npy",Y_pred_val)
    np.save(PATH_model +"y_pred_train.npy",Y_pred_train)   
    # calc logits
    logits_val = compute_logits(net, val_loader)
    logits_test = compute_logits(net, test_loader)
    logits_train = compute_logits(net, train_loader)
    # predict proba (after softmax)
    val_proba = softmax(logits_val, axis=1)
    test_proba = softmax(logits_test, axis=1)
    train_proba = softmax(logits_train, axis=1)
    np.save(PATH_model +"all_predictions_val.npy",val_proba)
    np.save(PATH_model +"all_predictions_test.npy",test_proba)
    np.save(PATH_model +"all_predictions_train.npy",train_proba)

In [45]:
for i in range(10):
    main('SignLanguage',i)

SignLanguage_net(
  (block1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): ReLU()
  )
  (block2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): ReLU()
  )
  (lin1): Linear(in_features=784, out_features=100, bias=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (drop1): Dropout(p=0.3, inplace=False)
  (lin2): Linear(in_features=100, out_features=24, bias=True)
)
preload state_dict
   testing progress: 96.33%  ......  loss: 0.0076 , acc: 0.9973

 test of the network on the 6926 test images: 99.74010973144672%
   testing progress: 96.33%  ......  loss: 0.0109 , acc: 0.9963

 val of the network on the 6926 test 

In [55]:

def preprosses_MNIST(dataset_name, shuffle_num):
    PATH = f'./{dataset_name}/{shuffle_num}/data/'

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    X_train = np.load(PATH + 'X_train.npy')
    X_test = np.load(PATH + 'X_test.npy')
    X_val = np.load(PATH + 'X_val.npy')
    y_train = np.load(PATH + 'y_train.npy')
    y_test = np.load(PATH + 'y_test.npy')
    y_val = np.load(PATH + 'y_val.npy')

    train_mean = X_train.mean() / 255.
    train_std = X_train.std() / 255.

    isRGB = "RGB" in dataset_name
    data = Data(X_train, X_test, X_val, y_train, y_test, y_val, len(set(y_train)), isRGB)

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])
    # Also use X_train in normalize since train/val sets should have same distribution
    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[train_mean], std=[train_std]),
    ])

    trainset = MNIST_from_array(data=X_train, label=y_train, transform=train_transform)
    valset = MNIST_from_array(data=X_val, label=y_val, transform=val_transform)
    testset = MNIST_from_array(data=X_test, label=y_test, transform=test_transform)

    batch_size = 64

    train_loader = DataLoader(dataset=trainset,
                              batch_size=batch_size,
                              shuffle=False)

    val_loader = DataLoader(dataset=valset,
                            batch_size=batch_size,
                            shuffle=False)

    test_loader = DataLoader(dataset=testset,
                             batch_size=batch_size,
                             shuffle=False)

    return train_loader, val_loader, test_loader, data



In [56]:

def main(dataset_name, shuffle_num):
    train_loader, val_loader, test_loader, data = preprosses_MNIST(dataset_name, shuffle_num)
    net = MNIST_net()

    num_epochs = 30
    learning_rate = 0.001

    if torch.cuda.is_available():
        net = net.cuda()
    print(net)
    print('=================================================================')

    # CrossEntropyLoss - Loss function
    criterion = nn.CrossEntropyLoss()  # loss function
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    PATH_model = f'./{dataset_name}/{shuffle_num}/pytorch/'
    # training and validating

    print('preload state_dict')
    net.load_state_dict(torch.load(PATH_model + 'model.pth'))

    # calc predictions
    Y_pred_test = model_testing(net, criterion, test_loader, 'test')
    Y_pred_val = model_testing(net, criterion, val_loader, 'val')
    Y_pred_train = model_testing(net, criterion, train_loader, 'train')
    np.save(PATH_model +"y_pred_test.npy",Y_pred_test)
    np.save(PATH_model +"y_pred_val.npy",Y_pred_val)
    np.save(PATH_model +"y_pred_train.npy",Y_pred_train)   
    # calc logits
    logits_val = compute_logits(net, val_loader)
    logits_test = compute_logits(net, test_loader)
    logits_train = compute_logits(net, train_loader)
    # predict proba (after softmax)
    val_proba = softmax(logits_val, axis=1)
    test_proba = softmax(logits_test, axis=1)
    train_proba = softmax(logits_train, axis=1)
    np.save(PATH_model +"all_predictions_val.npy",val_proba)
    np.save(PATH_model +"all_predictions_test.npy",test_proba)
    np.save(PATH_model +"all_predictions_train.npy",train_proba)


In [57]:
for i in range(10):
    main('MNIST',i)

MNIST_net(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Dropout(p=0.25, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    

# stability function per example:

In [2]:
def new_stability_calc(trainX,testX,train_y,test_y_pred,num_labels):
    '''
    Calculates the stability of the test set.
            Parameters:
                    trainX (List)
                    testX (List) 
                    train_y (List)
                    test_y_pred (list)
                    num_labels (Int)
            Returns:
                    stability(List)
    '''    
    time_lst = []
    same_nbrs=[]
    other_nbrs=[]
    for i in range(num_labels):
        idx_other=np.where(train_y!=i)
        other_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_other]))
        idx_same=np.where(train_y==i)
        same_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_same]))
    
    
    stability=np.array([-1.]*testX.shape[0])
    start = time.time()
    for i in range(testX.shape[0]):
        x=testX[i]
        pred_label=test_y_pred[i]
        
        dist1,idx1= same_nbrs[pred_label].kneighbors([x])
        dist2,idx2= other_nbrs[pred_label].kneighbors([x])

        stability[i]=(dist2-dist1)/2
    end = time.time()
    time_all=end-start
    ex_in_time=testX.shape[0]/time_all
    return stability,time_all,ex_in_time

In [ ]:
def load_model_pytorch(dataset_name, model_name, shuffle_num, isCalibrate=False):
    '''
    loads the model of specific shuffle
    '''
    data = load_data(dataset_name,shuffle_num)

    calc_dir = f'{dataset_name}/{shuffle_num}/{model_name}/'

    adder = "_calibrated" if isCalibrate else ""

    #all_predictions_val = np.load(calc_dir + f'all_predictions_val{adder}.npy', allow_pickle=True)
    #all_predictions_test = np.load(calc_dir + f'all_predictions_test{adder}.npy', allow_pickle=True)
    y_pred_test = np.load(calc_dir + f'y_pred_test{adder}.npy', allow_pickle=True)
    y_pred_val = np.load(calc_dir + f'y_pred_val{adder}.npy', allow_pickle=True)
    y_pred_train = np.load(calc_dir + f'y_pred_train{adder}.npy', allow_pickle=True)
    #all_predictions_train = np.load(calc_dir + f'all_predictions_train{adder}.npy', allow_pickle=True)

    return ModelInfo(data, y_pred_val, None, y_pred_test, None, y_pred_train,
                     None,dataset_name, model_name, shuffle_num, isCalibrate)




In [1]:
import numpy as np
import warnings
from utils import *
from calibrators import * 
import pickle
warnings.filterwarnings("ignore")
import torch
import sys
sys.path.append('../')
import time
import copy
from SLURM.sklearn_config import *
import time
import tensorflow as tf

In [ ]:
#max(2)
all_time=[]
for dataset_name in ['CIFAR_RGB']:#,'GTSRB_RGB']:
    for model_name in ['RF']:#,'GB']:#,'pytorch']:
        for shuffle in range(2,10):
            polling = torch.nn.MaxPool2d(2)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            model_info.data.X_train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),3,pixels,pixels)))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(torch.tensor(model_info.data.X_test.reshape((len(data.X_test),3,pixels,pixels)))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(torch.tensor(model_info.data.X_val.reshape((len(data.X_val),3,pixels,pixels)))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time